<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=301d77bdff1272c534696bc0d925c6c981e94a9917d099bfe2ee2b043a15d164
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-02 10:19:19
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -67.05 K (-0.47%)
Current PnL: -23.74 L (-15.55%)
CY Booked + Current PnL: -9.85 L (-6.45%)
-------------------
Total profit:  1.41 L
Total loss:  -25.14 L
-------------------
Total Booked + Current PnL: 17.13 L (13.64%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.78%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.15 L (62.75%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.25,-8.85,9.73,0.02,15481.0,-15455.0,159104.0,142.75,66.0,M-SC,13.06,234.0,-1.00,1.12,38.39,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.50,-16.40,19.73,0.09,16622.0,-16531.0,84246.0,95.52,39.0,M-SC,3.00,253.0,-0.99,0.59,10.68,OX40N,NTT,DURABLES
43,ITC,452.00,-0.35,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,43.0,X-LC,1.39,5.0,-0.13,1.39,3.90,X40,NTT,FMCG
51,MASFIN,398.61,1.07,-2.73,25.47,22.05,24275.0,-2670.0,95310.0,-15.99,58.0,H-SC,6.61,164.0,-0.11,0.67,38.54,XR,ATH,FINANCE
35,ICICIGI,2252.93,-1.64,4.50,15.02,20.19,27360.0,7839.0,182159.0,-18.14,43.0,X-MC,6.89,68.0,0.29,1.28,20.29,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,3.37,-27.76,108.73,50.79,110475.0,-39048.0,101605.0,-7.23,62.0,H-SC,6.42,151.0,-0.35,0.72,18.30,XR,ATH,IT
84,WHIRLPOOL,2270.00,2.15,-18.93,124.40,81.92,115775.0,-21731.0,93067.0,-52.45,24.0,M-SC,3.93,236.0,-0.19,0.66,10.18,XR,NTT,DURABLES
76,TMPV,600.00,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.68,3.22,XY24,NTT,AUTO
67,SIS,528.00,1.10,-25.05,61.79,21.26,51421.0,-27813.0,83219.0,1962.18,44.0,H-SC,3.86,166.0,-0.54,0.59,12.44,OX40N,NTT,MISC
5,ASIANPAINT,3460.25,1.09,1.32,19.37,20.95,49412.0,3326.0,255094.0,0.35,76.0,X-LC,16.44,31.0,0.07,1.80,37.43,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-7.39,-20.85,87.56,48.45,138960.0,-41803.0,158703.0,-26.53,13.0,X-MC,8.32,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
52,MEDANTA,1486.00,-3.08,0.82,23.06,24.08,29516.0,1045.0,127995.0,-7.41,40.0,X-SC,5.93,89.0,0.04,0.90,20.60,XY24,NTT,HEALTHCARE
56,QUESS,424.00,-1.90,-29.56,102.77,42.84,47061.0,-19213.0,45793.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
81,VALIANTORG,838.00,-1.90,-46.42,225.06,74.15,197270.0,-75953.0,87652.0,-257.83,34.0,H-SC,18.87,149.0,-0.39,0.62,12.36,XR,NTT,CHEMICALS
38,INDIAMART,4810.62,-1.84,-1.68,110.26,106.72,133700.0,-2077.0,121259.0,-51.39,35.0,H-SC,1.98,139.0,-0.02,0.85,21.30,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,0.37,-0.07,48.73,48.63,97371.0,-136.0,199818.0,-33.63,64.0,H-SC,15.12,136.0,-0.0,1.41,11.48,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.24,-5.19,119.28,107.91,166978.0,-7656.0,139988.0,-22.98,35.0,M-SC,10.96,216.0,-0.05,0.99,0.10,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.25,-8.85,9.73,0.02,15481.0,-15455.0,159104.0,142.75,66.0,M-SC,13.06,234.0,-1.00,1.12,38.39,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.50,-16.40,19.73,0.09,16622.0,-16531.0,84246.0,95.52,39.0,M-SC,3.00,253.0,-0.99,0.59,10.68,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.29,-23.54,48.40,13.47,99800.0,-63468.0,206199.0,-68.51,30.0,H-SC,1.98,158.0,-0.64,1.45,4.85,XY24,NTT,PAINTS
67,SIS,528.00,1.10,-25.05,61.79,21.26,51421.0,-27813.0,83219.0,1962.18,44.0,H-SC,3.86,166.0,-0.54,0.59,12.44,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.29,2.36,68.3,72.27,118462.0,3996.0,173444.0,-11.98,58.0,M-LC,3.55,99.0,0.03,1.22,8.68,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.29,2.36,68.30,72.27,118462.0,3996.0,173444.0,-11.98,58.0,M-LC,3.55,99.0,0.03,1.22,8.68,XR,NTT,IT
37,IEX,219.00,0.37,-0.07,48.73,48.63,97371.0,-136.0,199818.0,-33.63,64.0,H-SC,15.12,136.0,-0.00,1.41,11.48,XR,NTT,MISC
38,INDIAMART,4810.62,-1.84,-1.68,110.26,106.72,133700.0,-2077.0,121259.0,-51.39,35.0,H-SC,1.98,139.0,-0.02,0.85,21.30,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.20,-1.05,37.07,35.63,76870.0,-2210.0,207364.0,-14.97,43.0,H-MC,3.67,119.0,-0.03,1.46,15.44,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.24,-5.19,119.28,107.91,166978.0,-7656.0,139988.0,-22.98,35.0,M-SC,10.96,216.0,-0.05,0.99,0.10,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-7.39,-20.85,87.56,48.45,138960.0,-41803.0,158703.0,-26.53,13.0,X-MC,8.32,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.03,-37.82,117.43,35.20,93958.0,-48658.0,80012.0,4.84,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-0.37,-8.97,38.48,26.07,57358.0,-14680.0,149060.0,-32.90,27.0,X-MC,10.27,55.0,-0.26,1.05,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.31,-20.26,92.50,53.50,222566.0,-61139.0,240612.0,-61.46,31.0,X-MC,2.58,58.0,-0.27,1.69,6.87,XY24,NTT,FMCG
55,PGHH,17907.65,-0.34,-4.81,40.52,33.76,77458.0,-9660.0,191160.0,-32.91,33.0,X-MC,4.26,57.0,-0.12,1.35,0.14,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.35,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,43.0,X-LC,1.39,5.0,-0.13,1.39,3.90,X40,NTT,FMCG
3,ACC,3906.0,-0.19,-22.38,111.48,64.15,205904.0,-53251.0,184700.0,-54.57,48.0,X-SC,1.53,82.0,-0.26,1.30,3.61,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.49,0.89,23.69,24.79,60193.0,2231.0,254087.0,-3.69,46.0,X-SC,1.90,86.0,0.04,1.79,24.23,X40N,NTT,MISC
36,ICICIPRULI,790.0,-0.42,2.61,27.64,30.97,50126.0,4614.0,181352.0,-21.49,53.0,X-MC,2.17,75.0,0.09,1.28,15.50,X40,ATH,INSURANCE
76,TMPV,600.0,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.68,3.22,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.03,-37.82,117.43,35.20,93958.0,-48658.0,80012.0,4.84,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-7.39,-20.85,87.56,48.45,138960.0,-41803.0,158703.0,-26.53,13.0,X-MC,8.32,64.0,-0.30,1.12,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.37,-8.97,38.48,26.07,57358.0,-14680.0,149060.0,-32.90,27.0,X-MC,10.27,55.0,-0.26,1.05,0.00,X40,ATH,APPARELS
56,QUESS,424.00,-1.90,-29.56,102.77,42.84,47061.0,-19213.0,45793.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,-0.34,-4.81,40.52,33.76,77458.0,-9660.0,191160.0,-32.91,33.0,X-MC,4.26,57.0,-0.12,1.35,0.14,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.90,-29.56,102.77,42.84,47061.0,-19213.0,45793.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,0.00,-47.21,188.52,52.31,144461.0,-68531.0,76629.0,-43.76,36.0,X-SC,4.90,91.0,-0.47,0.54,2.57,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.03,-37.82,117.43,35.20,93958.0,-48658.0,80012.0,4.84,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-3.08,0.82,23.06,24.08,29516.0,1045.0,127995.0,-7.41,40.0,X-SC,5.93,89.0,0.04,0.90,20.60,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.39,-6.82,30.48,21.58,39801.0,-9558.0,130581.0,-53.04,34.0,X-MC,6.00,56.0,-0.24,0.92,15.61,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.11,-14.96,40.26,19.28,118445.0,-51755.0,294201.0,-25.89,57.0,X-LC,6.92,1.0,-0.44,2.07,8.76,X40,ATH,IT
41,INFY,2275.00,0.13,6.61,45.27,54.87,151002.0,20670.0,333558.0,-17.09,62.0,X-LC,2.49,2.0,0.14,2.35,15.44,X40,BTT,IT
76,TMPV,600.00,1.96,-13.79,64.93,42.18,154721.0,-38121.0,238289.0,-23.08,36.0,X-LC,2.29,3.0,-0.25,1.68,3.22,XY24,NTT,AUTO
82,VBL,671.64,-0.68,-2.87,39.68,35.67,121731.0,-9060.0,306782.0,-14.36,61.0,X-LC,5.45,4.0,-0.07,2.16,10.59,X40N,ATH,FMCG
43,ITC,452.00,-0.35,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,43.0,X-LC,1.39,5.0,-0.13,1.39,3.90,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.53,-37.12,111.35,32.89,54054.0,-28662.0,48544.0,-699.18,61.0,L-MC,5.92,259.0,-0.53,0.34,33.69,XR,NTT,BANKS
6,ASIANTILES,137.00,0.27,-15.05,113.86,81.67,90736.0,-14119.0,79691.0,7161.11,44.0,L-SC,19.03,271.0,-0.16,0.56,54.32,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.88,-12.77,59.97,39.54,52920.0,-12915.0,88244.0,-31.83,61.0,M-SC,6.62,220.0,-0.24,0.62,24.43,AR,ATH,AUTO
51,MASFIN,398.61,1.07,-2.73,25.47,22.05,24275.0,-2670.0,95310.0,-15.99,58.0,H-SC,6.61,164.0,-0.11,0.67,38.54,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.96,-19.71,53.23,23.03,76534.0,-35292.0,143779.0,55.70,48.0,H-SC,9.83,167.0,-0.46,1.01,42.66,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.25,-8.85,9.73,0.02,15481.0,-15455.0,159104.0,142.75,66.0,M-SC,13.06,234.0,-1.00,1.12,38.39,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,1.09,1.32,19.37,20.95,49412.0,3326.0,255094.0,0.35,76.0,X-LC,16.44,31.0,0.07,1.80,37.43,X40,ATH,PAINTS
13,BSOFT,831.70,3.37,-27.76,108.73,50.79,110475.0,-39048.0,101605.0,-7.23,62.0,H-SC,6.42,151.0,-0.35,0.72,18.30,XR,ATH,IT
50,LTIM,7201.88,-0.55,11.85,17.71,31.66,48761.0,29181.0,275332.0,7.45,66.0,H-LC,14.43,43.0,0.60,1.94,52.77,X200,ATH,IT
40,INDUSINDBK,1800.00,0.53,-37.12,111.35,32.89,54054.0,-28662.0,48544.0,-699.18,61.0,L-MC,5.92,259.0,-0.53,0.34,33.69,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.98
1,25,44.91
2,50,75.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.40
MC    29.01
LC    25.60
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.70
X40      21.26
X40N     11.95
XR        9.67
XY25      9.10
AR        9.07
OX40N     7.60
X200      1.94
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.52
X-LC    20.30
M-SC    11.88
X-SC     8.20
H-MC     4.77
H-LC     3.10
L-SC     1.39
M-MC     1.38
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.84,-5.14,39.15
IT,13.94,-14.22,73.18
FINANCE,9.43,-15.81,64.75
MISC,7.41,-23.56,74.18
ELECTRICAL,6.02,-9.68,49.85
PAINTS,5.95,-9.37,25.62
INSURANCE,4.39,-1.28,36.98
PHARMA,3.89,-0.97,33.28
AUTO,3.40,-19.17,70.26


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3101984.0,21
XR,1290312.0,13
AR,1283494.0,10
X40,1018090.0,14
X40N,751004.0,9
OX40N,713456.0,10
XY25,347652.0,6
SR,282232.0,2
MH,77623.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3580277.0,25
M-SC,1374209.0,15
X-MC,1283137.0,16
X-LC,880292.0,11
X-SC,753542.0,8
H-MC,400997.0,3
L-SC,263796.0,3
M-LC,118462.0,1
H-LC,116984.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230570.0      6
           AR         884164.0      5
M-SC       XY24       790528.0      6
H-SC       XR         776307.0      7
X-MC       X40        454434.0      7
X-LC       X40        388423.0      5
X-MC       XY24       381416.0      3
H-SC       OX40N      329381.0      4
M-SC       OX40N      296419.0      5
X-SC       X40N       295828.0      3
           XY24       282481.0      3
H-SC       SR         282232.0      2
X-MC       X40N       261594.0      4
X-LC       XY24       225186.0      2
H-MC       AR         209194.0      2
X-LC       X40N       193582.0      2
H-MC       XY24       191803.0      1
X-MC       XY25       185693.0      2
L-SC       XR         176140.0      2
X-SC       X40        175233.0      2
M-SC       XR         165349.0      2
           AR         121913.0      2
M-LC       XR         118462.0      1
L-SC       OX40N       87656.0      1
H-SC       MH          77623.0      1
X-LC       XY25        73101.0      2
H-LC       AR          68223.0      1
L-MC       XR          54054.0      1
M-MC       XY25        49279.0      1
H-LC       X200        48761.0      1
L-LC       XY25        39579.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
